In [1]:
## Create ouryelptipsbybusiness.csv and ouryelptipsbyusers.csv datasets 
##   from tips.json 
##   ouryelptipsbyuser is reduced by users who are in the ouryelusers.csv
##   ouryelptipsbybusiness is reduced by business who are in ouryelpbusiness.csv
##
## Source data is
##    "user_id": "UPw5DWs_b-e2JRBS-t37Ag",
##    "business_id": "VaKXUpmWTTWDKbpJ3aQdMw",
##    "text": "Great for watching games, ufc, and whatever else tickles yer fancy",
##    "date": "2014-03-27 03:51:24",
##    "compliment_count": 0
##
## Number of tips counted: 1223094

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import types
import re
#import ijson

import os
from datetime import datetime

%matplotlib inline
##

datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2019-05-28 11:15:27'

In [2]:
#setup environment
import os
cwd = os.getcwd()
print("Current Working Directory " , cwd)
os.chdir("C:/Users/Richpat/Documents/@syracuse coursework/@IST 718 Big Data Analytics (Python)/finalproject")
print('\nFiles in directory', os.listdir())
print('\n')
pd.show_versions(as_json=True) # True to shorten/flatten output
print ('\nStart date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Current Working Directory  c:\Users\Richpat\Documents\@syracuse coursework\@IST 718 Big Data Analytics (Python)\finalproject

Files in directory ['.ipynb_checkpoints', '16zpallnoagi.csv', '16zpdoc.doc', '2016_Historical_Business_File_Layout.xlsx', 'aclImdb_v1', 'aclImdb_v1.tar', 'aclImdb_v1.tar.gz', 'b.2.1_41473_project_checkpoint_1_clean.pdf', 'b.4.1_41474_project_checkpoint_2_clean.pdf', 'bCatTall.csv', 'business.json', 'BusinessCategoryCount.csv', 'checkin.json', 'Dataset_Challenge_Dataset_Agreement.pdf', 'DBHArizona164RestaurantsHighRisk.csv', 'DBHArizona164RestaurantsLowGT9.csv', 'DBHArizona164RestaurantsLowLT10.csv', 'DBHArizona164RestaurantsMediumGT6.csv', 'DBHArizona164RestaurantsMediumLT7.csv', 'DBHArizona164RestaurantsNARisk.csv', 'DBHNevada164RestaurantsHighRisk.csv', 'DBHNevada164RestaurantsLowRisk.csv', 'DBHNevada164RestaurantsMediumRisk.csv', 'DBHNevada164RestaurantsNARisk.csv', 'DBHRestaurants.csv', 'DBHRestaurantsSource.zip', 'DescribeYelpData.ipynb', 'DRT_IST718_FinalP

In [3]:
def readJSONreturnLIST (fn, verbosity):

    print ('Start readJSONreturnLIST for ', fn, 'at date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    with open(fn, encoding='utf-8') as json_file:      
        data = json_file.readlines()
        # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
        data = list(map(json.loads, data)) 
        
    print ('Type data', type(data), 'len data', len(data))
        
    print ('Start readJSONreturnLIST date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    return (data)

In [4]:
def descriptiveStatsNum(vals, key, title, verbose):
    x_plot=vals
    
    print ("Data Sample for ", title)
    print(pd.DataFrame.head(x_plot))
    
    ax = sns.distplot(x_plot)
    ax.axvline(x_plot.median(), lw=2.5, ls='dashed', color='black')
    ax.axvline(x_plot.mean(), lw=2.5, ls='dashed', color='red')
    plt.title(title + ' Distribution with Median (black) and Mean (red)')
    plt.show()

    print (title +' Descriptive Stats for ' + key)
    print (x_plot.describe())

    ax = sns.boxplot(x_plot, orient='v')
    plt.title(title + ' Boxplot')
    plt.show()
    
    return

In [5]:
#plot barchart from row/column input

def mybarchart(mbcmx, mbcx, mbcy, mbchowmany, mbctitle):
    # mbcmx innound rowxcol mx (row = groups to display, cols = values)
    # mbcx colname of x
    # mbcy colname of y
    # mbchowmany number of bars to display
    # mbctitle what is the title of the chart
    
    PltMX=mbcmx
    PltToShow=mbchowmany
    PltTitle=mbctitle + ': Top ' + str(PltToShow)

    print (PltTitle, ' ', PltMX.shape)
    #print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    #set PltMX (plot matrix) as input values; assumes input is in correct order
    PltMX=PltMX.head(PltToShow)

    #PltMX.columns = ['Count']
    PltMX[mbcx]=PltMX.index #use index as plot label
    PltMX['Label']=PltMX[mbcx]

    #prep labels
    bars=PltMX['Label']
    y_pos = np.arange(len(bars))
    height = PltMX[mbcy] #show 3 decimals if we are choosing the top 20

    #bars
    plt.bar(y_pos, height)

    #xlabels, ylabels, title
    plt.xticks(y_pos, bars, color='blue', rotation=90)
    plt.yticks(color='blue')
    plt.title(PltTitle)

    # Text on the top of each barplot
    for i in range(len(height)):
        plt.text(x = (i)-0.5 , y = height[i]+0.1, s = str(height[i]) , size = 7)
    
    #width of display 
    fig = plt.gcf()
    fig.set_size_inches( 12, 7)

    # Show graphic
    plt.show()

    return()

In [6]:
# read in full tips.json dataset from yelp

print ('Start date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
tips=readJSONreturnLIST ('tip.json', 2)
print ('End date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


Start date/time: 2019-05-28 11:15:31
Start readJSONreturnLIST for  tip.json at date/time 2019-05-28 11:15:31
Type data <class 'list'> len data 1223094
Start readJSONreturnLIST date/time 2019-05-28 11:15:36
End date/time: 2019-05-28 11:15:36


In [7]:
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
rNoTipsCnt=0
print ('Sample of user data')
print ('===================')
for tip in tips:
    if rNoTipsCnt==0:
        print (tip)
        print ('User', tip['user_id'])
        print ('Business', tip['business_id'])
        print (json.dumps(tip, indent=4))
    rNoTipsCnt+=1
print('Number of tips counted:', rNoTipsCnt)
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Date/time: 2019-05-28 11:15:36
Sample of user data
{'user_id': 'UPw5DWs_b-e2JRBS-t37Ag', 'business_id': 'VaKXUpmWTTWDKbpJ3aQdMw', 'text': 'Great for watching games, ufc, and whatever else tickles yer fancy', 'date': '2014-03-27 03:51:24', 'compliment_count': 0}
User UPw5DWs_b-e2JRBS-t37Ag
Business VaKXUpmWTTWDKbpJ3aQdMw
{
    "user_id": "UPw5DWs_b-e2JRBS-t37Ag",
    "business_id": "VaKXUpmWTTWDKbpJ3aQdMw",
    "text": "Great for watching games, ufc, and whatever else tickles yer fancy",
    "date": "2014-03-27 03:51:24",
    "compliment_count": 0
}
Number of tips counted: 1223094
Date/time: 2019-05-28 11:15:36


In [8]:
#convert TIPS to DF
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
TipsDF=pd.DataFrame(tips)
#TipsDF.index=usersDF['user_id']
#TipsDF.drop('user_id', axis=1, inplace=True)  

print("TipsDF data")
print("============")

#print("\n***** isna")
#print(TipsDF.isnull().sum())

print("\n***** dtypes")
print(TipsDF.dtypes)

print("\n***** head")
print(pd.DataFrame.head(TipsDF))

print("\n***** describe")
print(TipsDF.describe())

print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Date/time: 2019-05-28 11:15:36
TipsDF data

***** dtypes
business_id         object
compliment_count     int64
date                object
text                object
user_id             object
dtype: object

***** head
              business_id  compliment_count                 date  \
0  VaKXUpmWTTWDKbpJ3aQdMw                 0  2014-03-27 03:51:24   
1  OPiPeoJiv92rENwbq76orA                 0  2013-05-25 06:00:56   
2  5KheTjYPu1HcQzQFtm4_vw                 0  2011-12-26 01:46:17   
3  TkoyGi8J7YFjA6SbaRzrxg                 0  2014-03-23 21:32:49   
4  AkL6Ous6A1atZejfZXn1Bg                 0  2012-10-06 00:19:27   

                                                text                 user_id  
0  Great for watching games, ufc, and whatever el...  UPw5DWs_b-e2JRBS-t37Ag  
1  Happy Hour 2-4 daily with 1/2 price drinks and...  Ocha4kZBHb4JK0lOWvE0sg  
2  Good chips and salsa. Loud at times. Good serv...  jRyO2V1pA4CdVVqCIOPc1Q  
3  The setting and decoration here is amazing. Co...  FuT

In [9]:
OurBusinessesDF = pd.read_csv("ouryelpbusinesses.csv", sep="|")
OurBusinessesDF.index=OurBusinessesDF['business_id']
OurBusinessesDF.drop('business_id', axis=1, inplace=True)  

print("OurBusinessesDF data")
print("====================")

print("\n***** head")
print(pd.DataFrame.head(OurBusinessesDF))


OurBusinessesDF data

***** head
                        Unnamed: 0                       address  \
business_id                                                        
1Dfx3zM-rW4n-31KeC8sJg          11       2450 E Indian School Rd   
PZ-LZzSlhSe9utkQYU8pFg          17  1775 E Tropicana Ave, Ste 29   
LB6ZyCfUzeX9OLdunHhnOQ          30        8800 W Charleston Blvd   
HVPcIcqiJkrpD36xZFGN6g          34           3150 N Rainbow Blvd   
0_ohldeFwysbglrTLSGM4Q          36              15224 N 59th Ave   

                                                               attributes  \
business_id                                                                 
1Dfx3zM-rW4n-31KeC8sJg  {'RestaurantsTakeOut': 'True', 'BusinessParkin...   
PZ-LZzSlhSe9utkQYU8pFg  {'OutdoorSeating': 'False', 'BusinessAcceptsCr...   
LB6ZyCfUzeX9OLdunHhnOQ  {'BusinessAcceptsCreditCards': 'True', 'Restau...   
HVPcIcqiJkrpD36xZFGN6g  {'BusinessAcceptsCreditCards': 'True', 'ByAppo...   
0_ohldeFwysbglrTLSGM4Q  {'By

In [10]:
OurUsersDF = pd.read_csv("ouryelpusers.csv", sep=",")
OurUsersDF.index=OurUsersDF['user_id']
OurUsersDF.drop('user_id', axis=1, inplace=True)  

print("OurUsersDF data")
print("===============")

print("\n***** head")
print(pd.DataFrame.head(OurUsersDF))

OurUsersDF data

***** head
                        average_stars  compliment_cool  compliment_cute  \
user_id                                                                   
---1lKK3aKOuomHnwAkAow           3.92                9                1   
---PLwSf5gKdIoVnyRHgBA           4.00                0                0   
---udAKDsn0yQXmzbWQNSw           3.87                0                0   
--0LlX_UcypHnxW-4BUAdA           5.00                0                0   
--0WZ5gklOfbUIodJuKfaQ           4.20                0                0   

                        compliment_funny  compliment_hot  compliment_list  \
user_id                                                                     
---1lKK3aKOuomHnwAkAow                 9               2                0   
---PLwSf5gKdIoVnyRHgBA                 0               0                0   
---udAKDsn0yQXmzbWQNSw                 0               0                0   
--0LlX_UcypHnxW-4BUAdA                 0               0     

In [11]:
#Reduce the tips DF to have only those users that are in the OurUsersDF

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurTipsByUserDF=TipsDF.join(OurUsersDF, on='user_id',how='right')
print ("\nNumber of users on the OurUsersDF", len(OurUsersDF))
print ("\nNumber of tips on the TipsDF", len(TipsDF))
print ("\nNumber of tip/users on the OurTipsDF", len(OurTipsByUserDF))

print("OurTipsByUserDF data")
print("====================")

#print("\n***** isna")
#print(TipsDF.isnull().sum())

print("\n***** dtypes")
print(OurTipsByUserDF.dtypes)

print("\n***** head")
print(pd.DataFrame.head(OurTipsByUserDF))

print("\n***** describe")
print(OurTipsByUserDF.describe())

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
       


Date/time: 2019-05-28 11:15:47

Number of users on the OurUsersDF 455680

Number of tips on the TipsDF 1223094

Number of tip/users on the OurTipsDF 1042431
OurTipsByUserDF data

***** dtypes
business_id            object
compliment_count      float64
date                   object
text                   object
user_id                object
average_stars         float64
compliment_cool         int64
compliment_cute         int64
compliment_funny        int64
compliment_hot          int64
compliment_list         int64
compliment_more         int64
compliment_note         int64
compliment_photos       int64
compliment_plain        int64
compliment_profile      int64
compliment_writer       int64
cool                    int64
elite                  object
fans                    int64
friends                object
funny                   int64
name                   object
review_count            int64
useful                  int64
yelping_since          object
CatCnt                  int

In [12]:
#Reduce the User DF to have only those users that are in the OurReviewsDF

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurTipsByBusinessDF=TipsDF.join(OurBusinessesDF,on='business_id', how='right')
print ("\nNumber of businesses on the OurBusinessesDF", len(OurBusinessesDF))
print ("\nNumber of tips on the TipsDF", len(TipsDF))
print ("\nNumber of tip/businesses on the OurTipsByBusinessDF", len(OurTipsByBusinessDF))

print("OurTipsByBusinessDF data")
print("====================")

#print("\n***** isna")
#print(TipsDF.isnull().sum())

print("\n***** dtypes")
print(OurTipsByBusinessDF.dtypes)

print("\n***** head")
print(pd.DataFrame.head(OurTipsByBusinessDF))

print("\n***** describe")
print(OurTipsByBusinessDF.describe())

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


Date/time: 2019-05-28 11:16:12

Number of businesses on the OurBusinessesDF 19122

Number of tips on the TipsDF 1223094

Number of tip/businesses on the OurTipsByBusinessDF 225423
OurTipsByBusinessDF data

***** dtypes
business_id          object
compliment_count    float64
date                 object
text                 object
user_id              object
Unnamed: 0            int64
address              object
attributes           object
categories           object
city                 object
hours                object
is_open               int64
latitude            float64
longitude           float64
name                 object
postal_code         float64
review_count          int64
stars               float64
state                object
IsRestaurants          bool
IsShopping             bool
IsFood                 bool
IsHomeservices         bool
IsBeautyspas           bool
dtype: object

***** head
                   business_id  compliment_count                 date  \
3       T

In [13]:
#run descriptive stats on non-transformed data
#descriptiveStatsNum(OurUsersDF['review_count'], 'review_count', 'Count of Reviews by User', 2)



In [14]:
#ztmp=pd.DataFrame(OurUsersDF['review_count'])
#ztmp.index=OurUsersDF.index.astype(str)
#ztmp=ztmp.sort_values('review_count', ascending=False)
#print(pd.DataFrame.head(ztmp))

#mybarchart(ztmp, 'user_id', 'review_count', 50, 'Number of reviews by User')


In [15]:
print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurTipsByUserDF.to_csv("ouryelptipsbyuser.csv", sep=",", encoding = 'utf-8')
print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurTipsByBusinessDF.to_csv("ouryelptipsbybusiness.csv", sep=",", encoding = 'utf-8')
print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


Date/time: 2019-05-28 11:16:15

Date/time: 2019-05-28 11:18:24

Date/time: 2019-05-28 11:18:31
